# Labelling the data with visited_contact_page and time_spent_contact_page
To achieve this, loop through the visit data, and then loop through the visit paths. Check if each ID is the contact page, if so update visited_contact_page to True (1), and add the corresponding time_spent also.

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [2]:
generic = lambda x: literal_eval(x)

conv = {'url_id_path': generic,
        'seconds_spent_path': generic}

df = pd.read_csv('DATA/Data/final_cleaned_data/dataNoUnscrapedVisitsOrUnder20Sec.csv', converters=conv)

In [4]:
references = pd.read_csv('DATA/references/url_references_reduced.csv')

In [59]:
testDF = pd.read_csv('DATA/Data/testContact.csv', converters=conv)

In [41]:
# gets url_ids with contact or chatbox on them
def get_urls_with_prefix():
    contact_url = 'https://www.abp.nl/contact'
    chatbox_url = 'https://www.abp.nl/applicaties/chatbot'
    
    ids = []
    
    for index, row in references.iterrows():
        if contact_url in row.urls or chatbox_url in row.urls:
            ids.append(row.id)
    return ids

In [49]:
# Method which checks which visits visit contact page
def check_for_contact(contact_ids):
    visited_contact = np.zeros(len(df.index), dtype=int)
    time_spent = np.zeros(len(df.index), dtype=int)
    
    # Loop through rows in dataframe
    for index, row in df.iterrows():
        for index_url, url_id in enumerate(row.url_id_path):
            if url_id in contact_ids:
                visited_contact[index] = 1
                time_spent[index] = row.seconds_spent_path[index_url]
        if index % 100000 == 0:
            print(f"{index} rows complete...")
    print("DONE!")
    return visited_contact, time_spent
    

In [42]:
contact_ids = get_urls_with_prefix()

In [ ]:
visited_c, times_spent = check_for_contact(contact_ids)

In [53]:
df['visited_contact_page'] = list(visited_c)
df['time_spent_contact_page'] = list(times_spent)

In [56]:
df.to_csv('DATA/Data/dataContactLabels.csv', index=False)

In [ ]:
# Some small statistical analysis on the new data aspects
times_spent = []
contact_count = 0

for index, row in df.iterrows():
    if row.visited_contact_page == 1:
        contact_count += 1
        times_spent.append(row.time_spent_contact_page)
    if index % 100000 == 0:
        print(f"{index} rows complete...")
print("DONE!\n\n")

In [66]:
print(f"How many visits, visited the contact page: {contact_count}.")
print(f"Percentage of visits, visited the contact page: {(contact_count/len(df.index))*100}%")
print(f"Longest time spent on contact page: {max(times_spent)}.")
print(f"Shortest time spent on contact page: {min(times_spent)}.")
print(f"Average time spent on contact page: {sum(times_spent)/len(times_spent)}.")

How many visits, visited the contact page: 157394.
Percentage of visits, visited the contact page: 4.392419299474508%
Longest time spent on contact page: 4999.
Shortest time spent on contact page: 0.
Average time spent on contact page: 118.09793893032771.
